In [8]:
import csv
from collections import namedtuple


class ContextIterator:
    def __init__(self, fname):
        self._fname = fname
        self._file = None
        self._reader = None
        self._namedtuple = None

    def __iter__(self):
        return self

    def __next__(self):
        if self._file.closed:
            raise StopIteration
        else:
            return self._namedtuple(*next(self._reader))

    def sniff_dialect(self):
        sample = self._file.read(1000)
        self._file.seek(0)
        return csv.Sniffer().sniff(sample)

    def __enter__(self):
        print('---')
        print(f'{self._fname} opeing...')
        print('---')

        self._file = open(self._fname)

        dialect = self.sniff_dialect()
        self._reader = csv.reader(self._file, dialect)

        header_row = map(lambda s: s.lower(), next(self._reader))
        self._namedtuple = namedtuple('NamedContextIter', header_row)
        
        return self

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        print('---')
        print(f'{self._fname} closing...')
        print('---')

        self._file.close()
        return False


with ContextIterator('cars.csv') as f:
    print(next(f))
        

---
cars.csv opeing...
---
NamedContextIter(car='Chevrolet Chevelle Malibu', mpg='18.0', cylinders='8', displacement='307.0', horsepower='130.0', weight='3504.', acceleration='12.0', model='70', origin='US')
---
cars.csv closing...
---
